# Обучение сверточной нейросетевой модели для задачи классификации заболеваний дачных растений

Определим основные зависимости:
1. os - модуль Python для работы с файловой системой. Потребуется для поиска по структуре датасета;
2. tqdm - создание прогресс-баров в циклах для осущствления принципа ясного статуса системы;
3. matplotlib - библиотека для визуализации данных. Необходима для визуальных демонстраций различного назначения;
4. torch и  torchvision - основные библиотеки в экосистеме PyTorch для глубокого обучения
5. sklearn - библиотека для машинного обучения

In [ ]:
import os
from tqdm import tqdm
import matplotlib.pyplot as plt 

import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets
import torchinfo

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

## Подготовка данных для обучения модели

### 1. Источник данных
Укажем путь до сформированного набора данных в переменной dataset_dir

In [ ]:
dataset_dir = "./result_dataset/"

### 2. Нормализация
Опишем характер процедуры нормализации (normalize_transform) на основе сведений (mean и std) из предыдущей работы

In [ ]:
normalize_transform = torchvision.transforms.Compose([ 
    torchvision.transforms.Resize(size=(224, 224)),
    torchvision.transforms.ToTensor(), 
    torchvision.transforms.Normalize(
        mean=[0.42195199, 0.52360493, 0.31295609],
        std=[0.24439323, 0.25096216, 0.24255903]
    ),
]) 

### 3. Размер батча
Определяем размер батча BATCH_SIZE - параметра, который определяет сколько примеров данных обрабатывается за одну итерацию перед обновлением весов модели

In [ ]:
BATCH_SIZE = 24

### 4. Формирование выборок
На основе разделения данных по соответсвующим директориям формируем выборки Train, Validation и Test

In [ ]:
train_dir = os.path.join(dataset_dir, 'Train')
valid_dir = os.path.join(dataset_dir, 'Validation')
test_dir = os.path.join(dataset_dir, 'Test')

train_data = datasets.ImageFolder(train_dir, normalize_transform)
valid_data = datasets.ImageFolder(valid_dir, normalize_transform)
test_data = datasets.ImageFolder(test_dir, normalize_transform)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

print(train_data.classes)
print(train_data.class_to_idx)

## Выбор устройства обучения
Определим тип устройства, на основе которого будет проведено обучение модели. Приоритет отдается GPU при его доступности

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

## Объявление модели
Для обучения объявим модель (в данном случае, efficientNet) и определим структуру выходного слоя в качестве 5 нейронов, соотвествующих числу классов в выборке. После чего переведем модель на выборнное устройство

In [ ]:
output = len(train_data.classes)

model = torchvision.models.efficientnet_b3()
model.classifier[1] = torch.nn.Linear(in_features=1536, out_features=output)
model = model.to(device)

torchinfo.summary(model, (BATCH_SIZE, 3, 224, 224))

## Подготовка к обучению
Для формирования цикла обучения необходимо определить:
- функцию потерь, предназначенная для классификации, например, кросс-энтропия
- оптимизатор: AdamW - улучшенная версия Adam с правильной L2 регуляризацией (техника для предотвращения переобучения)
- планироващик скорости обучения: планировщик, уменщающий скорость обучения по экспоненциальному закону: y(n+1) = 0.91*y(n) (y(0) = a) или y = a * b ^ n, где a = 0.001, b = 0.91

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4) 
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(
    optimizer, 
    0.91
)

## Обучение
Определяем число эпох EPOCHS, префикс имени сохранения и вспомогательные структуры для визуализации процесса обучения

In [ ]:
EPOCHS = 1
SAVE_NAME = "enb3-adamw-res2"

train_loss = [] 
train_accuracy = []

valid_loss = []
valid_accuracy = []

learn_rate = []

best_valid_accuracy = 0

if not os.path.exists('./saves'):
    os.mkdir('./saves')

for epoch in range(EPOCHS): 

    # Процесс обучения на тренировочной выборке
    model.train() 
    
    running_train_loss = []  
    train_correct = 0    

    train_loop = tqdm(train_loader, leave=False)
    for batch, labels in train_loop: 

        batch = batch.to(device)
        labels = labels.to(device)

        outputs = model(batch)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1)    
        train_correct += (predicted == labels).sum().item()   

        running_train_loss.append(loss.item())

        train_loop.set_description(f'Epoch [{epoch + 1}/{EPOCHS}] - train')
        
    train_loss.append(sum(running_train_loss) / len(running_train_loss))
    train_accuracy.append(train_correct/ len(train_data))
 
    # Оценка обученности модели при помощи выборки Validation
    model.eval()

    running_valid_loss = []  
    valid_correct = 0    

    with torch.no_grad():

        valid_loop = tqdm(valid_loader, leave=False)
        for batch, labels in valid_loop:

            batch = batch.to(device)
            labels = labels.to(device)

            outputs = model(batch)
            loss = loss_fn(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)    
            valid_correct += (predicted == labels).sum().item()  

            running_valid_loss.append(loss.item())

            valid_loop.set_description(f'Epoch [{epoch + 1}/{EPOCHS}] - valid')

        valid_loss.append(sum(running_valid_loss) / len(running_valid_loss))
        valid_accuracy.append(valid_correct/ len(valid_data))

        lr = lr_scheduler._last_lr[0]
        learn_rate.append(lr)
        lr_scheduler.step()

    #Вывод результатов работы обучения модели на заданной эпохе
    print(f'Epoch [{epoch + 1}/{EPOCHS}]', end=' ')
    print(f"train_loss = {train_loss[-1]:.4f}", end=' ')
    print(f"train_acc = {train_accuracy[-1]:.4f}", end=' ')
    print(f"valid_loss = {valid_loss[-1]:.4f}", end=' ')
    print(f"valid_acc = {valid_accuracy[-1]:.4f}", end=' ')
    print(f"lr = {learn_rate[-1]}")

    target_accuracy = valid_accuracy[-1]

    #Сохранение более совершенной модели
    if (target_accuracy > best_valid_accuracy):

        print('Saving model...', end=' ')
        torch.save(model, f'./saves/{SAVE_NAME}-{(target_accuracy * 100):.0f}-{epoch+1}.pt')
        print('...Model saved')

        best_valid_accuracy = target_accuracy  


## Анализ обучения

### 1. Визулизация процесса обучени
Оторажаем на графиках, как изменялись ошибка и точность модели на Train и Validation выборках, а также скорость обучения с каждой эпохой.

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(2, 3, 1)
plt.plot(range(1,len(train_loss)+1), train_loss, label="Training Loss") 
plt.plot(range(1,len(valid_loss)+1), valid_loss, label="Valid Loss") 
plt.xlabel("Number of epochs") 
plt.ylabel("Loss") 
plt.legend(loc='upper right')

plt.subplot(2, 3, 2)
plt.plot(range(1,len(train_accuracy)+1), train_accuracy, label="Training Accuracy")
plt.plot(range(1,len(valid_accuracy)+1), valid_accuracy, label="Valid Accuracy")
plt.xlabel("Number of epochs") 
plt.ylabel("Accuracy") 
plt.legend(loc='lower right')

plt.subplot(2, 3, 4)
plt.plot(range(1,len(learn_rate)+1), learn_rate, label="Learning Rate")
plt.xlabel("Number of epochs") 
plt.ylabel("Learning Rate") 
plt.legend(loc='upper right')
plt.show()

### 2. Проверка на тестовых данных
Для оценки модели на тестовых данных произведем расчет таких показетелей как:
- Precision - показывает, насколько можно доверять модели, когда она предсказывает положительный класс. Отражает долю правильно предсказанных положительных примеров среди всех примеров;
- Recall - показывает, какую долю всех реально положительных примеров модель смогла обнаружить и правильно классифицировать;
- F1-score - это гармоническое среднее между Precision и Recall.

In [ ]:
model.eval()
test_correst = 0

y_true = []
y_pred =  []
with torch.no_grad():

    for batch, labels in test_loader:

        batch = batch.to(device)
        labels = labels.to(device)

        outputs = model(batch)
        _, predicted = torch.max(outputs.data, 1)    


        y_true += [i.item() for i in labels]
        y_pred += [i.item() for i in predicted]

        test_correst += (predicted == labels).sum().item()  

print(classification_report(y_true, y_pred))

Также полученные численные метрики удобно представить в виде матрицы ошибок

In [ ]:
cm = confusion_matrix(y_true, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues', values_format='d')
plt.show()

## Подготовка модели к эксплуатации
Для того, чтобы подготовить модель для эксплуатации на модильном устройстве, необходимо:
1. Перевести модель в режим вывода;
2. Перевести модель на CPU (так как исполнение будет происходит локально на мобильном устройстве);
3. Указать размерность  входного тензора;
4. Выполни трассировние - создать оптимизированную версию модели;
5. Сохранить модель в нужно место по пути PATH;

In [ ]:
PATH = "./ef-net-b3-mobile.pt"

model.eval()
model.cpu()

example_input = torch.randn(1, 3, 224, 224) 
traced_script_module = torch.jit.trace(model, example_input)

traced_script_module.save(PATH)